In [9]:
import os
from glob import glob
import json
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
from IPython.display import Image, display
import time
import numpy as np

import cv2
from collections import defaultdict

In [10]:
scatter = pd.read_csv('/Users/minsukchang/Research/ChartDataset/scatter/scatters.csv')


def real_corr(filename):
    direction, _, corr, exp = filename.split('_')
    row = scatter[(scatter['scatter_type'] == direction) & (scatter['corr'] == int(corr)) & (scatter['exp'] == int(exp))]
    if row.shape[0] == 0:
        print(filename)
    return row['corr_computed'].values[0]

In [11]:
labels = [
    # Neutral
    ['As the usage of internet increases, so does the homicide rate in the city.', 'Increasing internet usage', 'Higher homicide rate in the city'],
    ['People who eat more cheese, tend to be better at dancing.', 'Eating more cheese', 'Better at dancing'],
    ['The more students wear glasses, the later the gym closes on campus.', 'More students wearing glasses', 'Gym closing later on campus'],
    ['A city with more lawyers, tends to have more trees.', 'Having more lawyers in a city', 'Having more trees in the city'],

    # Positive
    ['The more often students eat breakfast, the higher their GPAs are.', 'Eating breakfast more often', 'Having a higher GPA'],
    ['A worker with a longer commute, tends to be more stressed.', 'Commuting for a longer time', 'Being more stressed'],
    ['People who sleep more, tend to be happier with their lives.', 'Sleeping more', 'Being happier with life'],
    ['As the number of environmental regulations increases, so does the air quality in the city.', 'Having more environmental regulations', 'Improving air quality in the city'],

    # New Neutral
    ['The more people buy socks, the more pigeons appear in the park.', 'Buying more socks', 'Seeing more pigeons in the park'],
    ['As the number of cats in the city increases, the library\'s carpet gets replaced more often.', 'Number of cats in city', 'Library carpet replacement frequency'],
    # ['The more times people sneeze, the more clouds appear in the sky.', 'Sneezing frequency', 'Cloud count in the sky'],
    # ['When students buy more notebooks, the football team wins more games.', 'Notebook purchases by students', 'More football team wins'],

    # Semi-positive
    ['The more drivers wear seatbelts, the more survivors there are in accidents.', 'Percentage of drivers wearing seatbelts', 'More accident survivors'],
    # ['The more rain boots people buy, the higher the number of umbrellas sold.', 'Rain boot sales', 'Umbrella sales'],
    # ['When people drink more coffee, they tend to search more often for productivity tips online.', 'Coffee consumption', 'Search frequency for productivity tips'],
    ['The more probiotic yogurt people buy, the more toilet paper sales go up.', 'Probiotic yogurt sales', 'Toilet paper sales'],
]

In [12]:
def generate_base_components():
    return {
        "bubble": {
            "type": "react-component",
            "path": "gaze/assets/bubble.jsx",
            "response": [
                {
                    "id": "answer",
                    "prompt": "",
                    "required": True,
                    "location": "sidebar",
                    "type": "reactive"
                },
            ],
            "parameters": {
                "ratio": 0.08,
            },
            "instructionLocation": "belowStimulus",
            "nextButtonLocation": "belowStimulus"
        },
        "plain": {
            "type": "react-component",
            "path": "gaze/assets/plain.jsx",
            "response": [
                {
                    "id": "answer",
                    "prompt": "",
                    "required": True,
                    "location": "sidebar",
                    "type": "reactive"
                },
            ],
            "instructionLocation": "belowStimulus",
            "nextButtonLocation": "belowStimulus"
        },
        "draw": {
            "type": "react-component",
            "path": "gaze/assets/draw.jsx",
            "response": [
                {
                    "id": "answer",
                    "prompt": "",
                    "required": True,
                    "location": "sidebar",
                    "type": "reactive"
                },
            ],
            "instructionLocation": "belowStimulus",
            "nextButtonLocation": "belowStimulus"
        }
    }

In [13]:
def create_default_components():
    return {
        "phase1_intro": {
            "type": "markdown",
            "path": "gaze/assets/phase1_intro.md",
            "response": []
        },
        "phase2_intro": {
            "type": "markdown",
            "path": "gaze/assets/phase2_intro.md",
            "response": []
        },
        "phase3_intro": {
            "type": "markdown",
            "path": "gaze/assets/phase3_intro.md",
            "response": []
        },
        "consent": {
            "type": "markdown",
            "path": "shared/consent.md",
            "nextButtonText": "I agree",
            "response": []
        },
        "introduction": {
            "type": "markdown",
            "path": "gaze/assets/introduction.md",
            "response": []  
        },
        "demographics": {
            "type": "markdown",
            "path": "shared/blank.md",
            "response": [
                {
                    "id": "gender",
                    "prompt": "What is your **gender**?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "withOther": True,
                    "options": [
                        "Woman",
                        "Man",
                        "Prefer not to say"
                    ],
                    "withDivider": True

                },
                {
                    "id": "age",
                    "prompt": "What is your **age**?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "Under 18 years",
                        "18-24 years",
                        "25-34 years",
                        "35-44 years",
                        "45-54 years",
                        "55-64 years",
                        "65 years or older",
                        "Prefer not to say"
                    ],
                    "withDivider": True
                },
                {
                    "id": "education",
                    "prompt": "What is the **highest degree or level of education** you have completed?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "withOther": True,
                    "options": [
                        "Less than high school",
                        "High school diploma or equivalent",
                        "Bachelor's degree or equivalent",
                        "Master's degree or equivalent",
                        "Doctoral degree or equivalent"
                    ],
                    "withDivider": True
                }
            ]
        },
    }

def create_phase1_components():
    components = {}
    for corr in [1, 3, 5, 7]:
        for i in range(1, 7): # for phase 1
            for direction in ["pos", "neg"]:
                components[f"plain_{corr}_{i}_{direction}"] = {
                    "baseComponent": "plain",
                    "parameters": {
                        "image": f"https://raw.githubusercontent.com/jangsus1/ChartQA/main/scatter/{direction}_scatter_{corr}_{i}.png",
                        "example": False,
                        "correlation": real_corr(f"{direction}_scatter_{corr}_{i}"),
                        "seconds": 5,
                    }
                }
    return components


def create_phase2_components():
    components = defaultdict(dict)
    for corr in [1, 3, 5, 7]:
        for i in range(6, 8):  # for phase 2
            for direction in ["pos"]:
                for label_idx, label in enumerate(labels):
                    label, x, y = label
                    components[label][f"phase2_{corr}_{i}_{direction}_{label_idx}"] = {
                        "baseComponent": "bubble",
                        "parameters": {
                            "image": f"https://raw.githubusercontent.com/jangsus1/ChartQA/main/scatter/{direction}_scatter_{corr}_{i}.png",
                            "example": False,
                            "correlation": real_corr(f"{direction}_scatter_{corr}_{i}"),
                            "label": label,
                            "X": x,
                            "Y": y,
                            "corr": corr,
                            "exp": i,
                            "direction": direction,
                            "seconds": 7,
                        }
                    }
    return components

def create_phase3_components():
    components = {}
    for label_idx, label in enumerate(labels):
        label, x, y = label
        components[f"phase3_{label_idx}"] = {
            "baseComponent": "draw",
            "parameters": {
                "image": "https://raw.githubusercontent.com/jangsus1/ChartQA/main/scatter/empty_scatter.png",
                "label": label,
                "X": x,
                "Y": y,
            }
        }
    return components

def sequence_generator(phase1_components, phase2_components, phase3_components):
    groups = [{
        "id": label[:30],
        "order": "latinSquare",
        "numSamples": 1,
        "components": list(phase2_components[label].keys())
    } for label in phase2_components.keys()]
    
    sequence = {
        "order": "fixed",
        "components": [
            # "welcome",
            "consent",
            "demographics",
            "introduction",
            "phase1_intro",
            {
                "id": "phase1",
                "order": "latinSquare",
                "components": list(phase1_components.keys())
            },
            "phase2_intro",
            {
                "id": "phase2",
                "order": "latinSquare",
                "components": groups
            },
            "phase3_intro",
            {
                "id": "phase3",
                "order": "latinSquare",
                "components": list(phase3_components.keys())
            }
        ]
    }
    return sequence

In [14]:
default_components = create_default_components()
phase1_components = create_phase1_components()
phase2_components = create_phase2_components()
phase3_components = create_phase3_components()
components = default_components | phase3_components | phase1_components
for component in phase2_components.values():
    components |= component

sequence = sequence_generator(phase1_components, phase2_components, phase3_components)
baseComponents = generate_base_components()
print(f"Total number of components: {len(components)}")

Total number of components: 162


In [15]:
prolificRedirection = "https://app.prolific.com/submissions/complete?cc=C1DEBJ8K"

In [16]:
with open("config.json", "r") as f:
    config = json.load(f)
config['uiConfig']['studyEndMsg'] = f"**Thank you for completing the study. You may click this link and return to Prolific**: [{prolificRedirection}]({prolificRedirection})"
config['components'] = components
config['sequence'] = sequence
config['baseComponents'] = baseComponents
with open("config.json", "w") as f:
    json.dump(config, f, indent=4)